# Feedforward Neural Network with PyTorch

## About Feedforward network 

#### Logistic Regression Transition to Neural Networks 

##### Logistic Regression Review 

In [3]:
import torch 
import torch.nn as nn

In [4]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        out = self.linear(x)
        return out

In [5]:
input_dim = 28*28
output_dim = 10

model = LogisticRegressionModel(input_dim, output_dim)

In [6]:
print(model)

LogisticRegressionModel(
  (linear): Linear(in_features=784, out_features=10, bias=True)
)


#### Logistic Regression Problems 

* Can represent linear function well
    * y =2x+3
    * y = x1+x2
    
* Can't represent non-linear function 
    * y=x1x2
     

In [8]:
from IPython.display import display, Math, Latex

### Introducing Non-linear Function

* Function : Takes a number & perform mathematical operation
* Common Types of Non-linearity
    * ReLus(Rectified Linear Units)
    * Sigmoid
    * Tanh
    



** Sigmoid (Logistic) **
* $ \sigma(x)=\frac{1}{1+e^{-x}} $
* Input Number -->[0,1]
    * Large negative number -->0
    * Large positive number -->1
* Cons:
    1. Activation saturates at 0 or 1 with gradients=0
        * No signal to update weights --> cannot learn
        * Solution : Have to carefully initialize weights to prevent this
    2. Output not centered around 0 
        * If output always positive --> gradients always positive or negative --> ** bad for gradient updats **
        
** Tanh **

* $ tanh(x)=2\sigma(2x)-1 $ 
    * A scaled sigmoid function
* Input Number -->[-1,1]
* Cons :
    1. Activation saturates at 0 or 1 with gradients=0
        * No signal to update weights --> cannot learn
        * Solution : Have to carefully initialize weights to prevent this
        
        
** ReLUs **

* $ f(x) =max(0,x) $
* Pros:
    1. Accelerates convergence --> ** Train faster **
    2. ** Less computationally expensive operation ** compared to Sigmoid/Tanh exponentials 
    * A scaled sigmoid function

* Cons :
    1. Many ReLU units * die*--> ** gradient** =0 forever  
        * No signal to update weights --> cannot learn
        * Solution : carefully choosing learning rate

# Building a Feedforward Neural Network with PyTorch 


##### Steps
* Step 1: Load Dataset
* Step 2: Make Dataset iterable
* Step 3: Make Model Class
* Step 4: Instantiate Model Class
* Step 5: Instantiate Loss Class
* Step 6: Instantiate Optimizer Class
* Step 7: Instantiate Train Model


#### Step 1 : Loading MNIST Dataset 

In [9]:
#importing dependencies
import torch
import torch.nn as nn
import torchvision.transforms as transforms 
import torchvision.datasets as dsets 
from torch.autograd import Variable 

In [10]:
# create the train and tets dataset 
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

#### STEP 2: MAKING DATASET ITERABLE


In [11]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

#### STEP 3: CREATE MODEL CLASS

In [21]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function 1: 784 --> 100
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity 1
        self.relu1 = nn.ReLU()
        
        # Linear function 2: 100 --> 100
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 2
        self.relu2 = nn.ReLU()
    
    def forward(self, x):
        # Linear function 1
        out = self.fc1(x)
        # Non-linearity 1
        out = self.relu1(out)
        
        # Linear function 2
        out = self.fc2(out)
        # Non-linearity 2
        out = self.relu2(out)
        return out

#### Step 4 :  Instantiate Model Class

* Input Dimension :
    * Size of image
    * 28*28 = 784
* Output Dimension : 10
    * 0,1,2,3,4,5,6,7,8,9
    
* Hidden Dimension : 100
    * can be any number
    * Similar term
        * Number of neurons
        * number of non-linear activation functions

In [13]:
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

#### Step 5 :  Instantiate Loss Class

* Feedforward Neural Network : **Cross Entropy Loss**
    * Logistic Regression : Cross Entropy Loss
    * Linear Regression : MSE 

In [14]:
criterion = nn.CrossEntropyLoss()

#### STEP 6: INSTANTIATE OPTIMIZER CLASS

#### Instantiate Optimizer Class

* Simplified Equation :$\Theta = \Theta -\eta .*\bigtriangledown _{\Theta }$
    * $\Theta$ : parameter(Our Variables)
    * $\eta$ : learning rate
    * $\bigtriangledown _{\Theta }$ : parameter's gradients
    
* Even Simpler equation:
    * parameters  = parameters-learning_rate * parameter_gradients
    * At every iteration , we update our model parameters
    

In [15]:
learning_rate = 0.001

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#### Parameters In-Depth

In [16]:
print(model.parameters())

print(len(list(model.parameters())))

# Hidden Layers Parameters
print(list(model.parameters())[0].size())

# FC1 Bias  Parameters
print(list(model.parameters())[1].size())

# FC2 Bias  Parameters
print(list(model.parameters())[2].size())

# FC2 Bias  Parameters
print(list(model.parameters())[3].size())

<generator object Module.parameters at 0x0000020A38165A98>
4
torch.Size([100, 784])
torch.Size([100])
torch.Size([100, 100])
torch.Size([100])


#### STEP 7: TRAIN THE MODEL

* Process
    1. Convert input/labels to variables
    2. Clear gradient buffets
    3. Get Output given inputs
    4. Get loss
    5. Get gradient w.r.t to parameters
    6. update parameters using gradients
        * parameters = parameters-learning_rate * parameter_gradients
    7. Repeat

In [35]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
    
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter %500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
               
                images = Variable(images.view(-1, 28*28))
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('%Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))        

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


%Iteration: 500. Loss: 3.085878372192383. Accuracy: 32
%Iteration: 1000. Loss: 2.594388723373413. Accuracy: 57
%Iteration: 1500. Loss: 2.4496636390686035. Accuracy: 61
%Iteration: 2000. Loss: 2.014299154281616. Accuracy: 63
%Iteration: 2500. Loss: 1.660508632659912. Accuracy: 66
%Iteration: 3000. Loss: 1.4427804946899414. Accuracy: 72


# More Feedforward Neural Network Models in PyTorch 

In [36]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

'''
STEP 1: LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function 1: 784 --> 100
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity 1
        self.relu1 = nn.ReLU()
        
        # Linear function 2: 100 --> 100
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 2
        self.relu2 = nn.ReLU()
        
        # Linear function 3: 100 --> 100
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 3
        self.relu3 = nn.ReLU()
        
        # Linear function 4 (readout): 100 --> 10
        self.fc4 = nn.Linear(hidden_dim, output_dim)  
    
    def forward(self, x):
        # Linear function 1
        out = self.fc1(x)
        # Non-linearity 1
        out = self.relu1(out)
        
        # Linear function 2
        out = self.fc2(out)
        # Non-linearity 2
        out = self.relu2(out)
        
        # Linear function 2
        out = self.fc3(out)
        # Non-linearity 2
        out = self.relu3(out)
        
        # Linear function 4 (readout)
        out = self.fc4(out)
        return out
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
    
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter %500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
               
                images = Variable(images.view(-1, 28*28))
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('%Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:153: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


%Iteration: 500. Loss: 0.32540589570999146. Accuracy: 89
%Iteration: 1000. Loss: 0.18222180008888245. Accuracy: 93
%Iteration: 1500. Loss: 0.2022702842950821. Accuracy: 94
%Iteration: 2000. Loss: 0.08823994547128677. Accuracy: 96
%Iteration: 2500. Loss: 0.10647334158420563. Accuracy: 96
%Iteration: 3000. Loss: 0.1004876121878624. Accuracy: 97
